In [1]:
%matplotlib inline

import networkx as nx
from tqdm import tqdm
from matplotlib import pyplot as plt

from graph_generator import grid_2d, add_p_and_delta
from utils import infeciton_time2weight
from plot_utils import plot_snapshot
from ic import make_partial_cascade, sample_graph_from_infection
from collections import defaultdict
from core import normalize_mu
from synthetic_data import load_data_by_gtype

In [2]:
if False:
    p = 0.7
    g = grid_2d(10)
    g = add_p_and_delta(g, p, 1)
    pos = {n: np.array(n) for n in g.nodes()}
else:
    g = load_data_by_gtype('kr-hier', '10-10')[0]

In [3]:
def build_reward_table(g, n_rounds=100):
    """
    return two tables.
    
    table 1 maps (s, u, v) to real number
    s: source
    u: earlier infected node
    v: later infected node
    
    table 2 maps (s, u) to real number, the probability that u is not infected given s
    """
    tbl1 = defaultdict(float)
    tbl2 = defaultdict(float)
    increase = 1 / n_rounds
    for i in tqdm(range(n_rounds)):
        sampled_g = sample_graph_from_infection(g)
        sp_path = nx.shortest_path(sampled_g)
        for s in g.nodes_iter():
            for q in g.nodes_iter():
                try:
                    path = sp_path[s][q]
                    if len(path) >= 2:
                        tbl1[(s, path[-2], q)] += increase                        
                except KeyError:
                    tbl2[(s, q)] += increase
    return tbl1, tbl2

In [ ]:
tbl1, tbl2 = build_reward_table(g)

In [ ]:
def reward_by_infection_direction(g, q, u, n_rounds=100):
    """for each node as source, calculate 
    the fraction of cascades in which cascade goes from u to q"""
    reward = {n: 0 for n in g.nodes_iter()}

    for i in range(n_rounds):
        sampled_g = sample_graph_from_infection(g)
        for s in g.nodes_iter():
            try:
                for path in nx.all_shortest_paths(sampled_g, source=s, target=q):
                    if u in path:
                        reward[s] += 1
                        break
            except nx.NetworkXNoPath:
                pass

    for n in g.nodes_iter():
        reward[n] /= n_rounds
    return reward

def reward_by_uninfected_node(g, q, n_rounds=100):
    reward = {n: 0 for n in g.nodes_iter()}

    for i in range(n_rounds):
        sampled_g = sample_graph_from_infection(g)
        for s in g.nodes_iter():
            try:
                nx.shortest_path(sampled_g, source=s, target=q)
            except nx.NetworkXNoPath:  # uninfected
                reward[s] += 1
    for n in g.nodes_iter():
        reward[n] /= n_rounds
    return reward

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
reward = reward_by_infection_direction(g, q, u)
plot_snapshot(g, pos, reward, query_node=q, ax=ax[0, 0])
ax[1, 0].hist(list(reward.values()))
ax[0, 0].set_title('correct')

reward = {n: tbl1[(n, u, q)] for n in g.nodes_iter()}
plot_snapshot(g, pos, reward, query_node=q, ax=ax[0, 1])
ax[1, 1].hist(list(reward.values()))
ax[0, 1].set_title('actual')

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
reward = reward_by_uninfected_node(g, q)
plot_snapshot(g, pos, reward, query_node=q, ax=ax[0, 0])
ax[1, 0].hist(list(reward.values()))
ax[0, 0].set_title('correct')

reward = {n: tbl2[(n, q)] for n in g.nodes_iter()}
plot_snapshot(g, pos, reward, query_node=q, ax=ax[0, 1])
ax[1, 1].hist(list(reward.values()))
ax[0, 1].set_title('actual')

In [14]:
def median_node(g, mu, sp_len):
    def sum_of_weighted_dist(q):
        mus = np.array([mu[v] for v in g.nodes_iter()])
        lens = np.array([sp_len[q][v] for v in g.nodes_iter()])
        return np.sum(mus * lens)

    return min(g.nodes_iter(), key=sum_of_weighted_dist)

In [12]:
def mwu_by_infection_direction(g, obs_nodes, infection_times, source,
                               direction_reward_table=None,
                               inf_reward_table=None,
                               save_logs=False,
                               debug=False):
    mu = {n: 1 for n in g.nodes_iter()}
    sp_len = nx.shortest_path_length(g, weight='d')
    centroids = []
    queried_nodes = set(obs_nodes)
    while True:
        if len(queried_nodes) == g.number_of_nodes():
            print("no more queries to go")
            break
        q = median_node(g, mu, sp_len)
        queried_nodes.add(q)
        
        if debug:
            print('query node: {}'.format(q))
        
        if save_logs:
            centroids.append(q)
        found_source = True
        if np.isinf(infection_times[q]):
            found_source = False
            if inf_reward_table:
                reward = {n: inf_reward_table[(n, q)] for n in g.nodes_iter()}
            else:
                reward = reward_by_uninfected_node(g, q)
        else:
            for u in g.neighbors(q):
                if infection_times[u] < infection_times[q]:
                    queried_nodes.add(u)
                    if direction_reward_table:
                        reward = {n: direction_reward_table[(n, u, q)] for n in g.nodes_iter()}
                    else:
                        reward = reward_by_infection_direction(g, q, u)                    
                    found_source = False
                    break
        if found_source:
            assert source == q
            break
            
        for n in g.nodes_iter():
            mu[n] *= reward[n]
        mu = normalize_mu(mu)
    return len(queried_nodes - obs_nodes)

In [8]:
def experiment_mwu_multiple_rounds(g,
                                   dir_tbl, inf_tbl,
                                   fraction=0.05,
                                   sampling_method='late_nodes',
                                   rounds=100):
    counts = []
    for i in tqdm(range(rounds)):
        source, obs_nodes, infection_times, tree = make_partial_cascade(
            g, fraction, sampling_method=sampling_method)
        query_count = mwu_by_infection_direction(
            g, obs_nodes, infection_times, source,
            direction_reward_table=dir_tbl,
            inf_reward_table=inf_tbl,
            save_logs=False)
        counts.append(query_count)
    return counts

In [4]:
dir_tbl, inf_tbl = build_reward_table(g)

100%|██████████| 100/100 [07:56<00:00,  5.31s/it]


In [15]:
counts = experiment_mwu_multiple_rounds(g, dir_tbl, inf_tbl)

100%|██████████| 100/100 [11:29<00:00,  6.61s/it]


In [17]:
g.number_of_nodes()

1011

In [16]:
from experiment_utils import counts_to_stat
counts_to_stat(counts)

{'25%': 6.0,
 '50%': 7.0,
 '75%': 9.0,
 'count': 100.0,
 'max': 28.0,
 'mean': 7.9299999999999997,
 'min': 3.0,
 'std': 3.2760378878187977}